In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

In [2]:
df = pd.read_csv("../archivos/cod_bikes.csv", index_col = 0)
df.head()

,estacion,año,dias_festivos,temperatura,humedad,velocidad_viento,clientes_casuales,clientes_registrados,dia_semana_domingo,dia_semana_jueves,...,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,mes_10,mes_11,mes_12
0,0,0,1,14.110847,80.5833,10.749882,331,654,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,14.902598,69.6087,16.652113,131,670,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,8.050924,43.7273,16.636703,120,1229,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,8.200000,59.0435,10.739832,108,1454,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,9.305237,43.6957,12.522300,82,1518,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.drop(['clientes_registrados','estacion'], axis=1,inplace=True)

In [4]:
X = df.drop("clientes_casuales", axis = 1)
y = df["clientes_casuales"]

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [6]:
param = {"max_depth": [2,4,6], # teniendo en cuenta que teníamos overfitting tendremos que reducir la profundidad del modelo, la nuestra anterior era de 17. Bajaremos mucho este valor ya que teníamos un overfitting muy claro
        "max_features": [1,2,3,4],# calculamos en celdas anteriores, probaremos a hacer el modelo como una variable, 2, 3 y 4. Ponemos como límite el 4 ya que es el resultado de la raiz cuadrada. 
        # estos dos hiperparámetros son más difíciles de definir, pero usualmente se suelen elegir los siguientes valores
        "min_samples_split": [10, 20,50, 100],
        "min_samples_leaf": [10,20, 50,100]} 

In [7]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") 

In [8]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 20, 50, 100],
                         'min_samples_split': [10, 20, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [9]:
bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=6, max_features=4, min_samples_leaf=10,
                      min_samples_split=20)

In [10]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [11]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [21]:
dt_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest enc Casual")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,325.452573,201598.412748,448.997119,0.580374,test,Random Forest enc Casual
1,322.960756,197942.659479,444.907473,0.576780,train,Random Forest enc Casual


In [22]:
dt_results.to_csv('../archivos/rf_enc_casual.csv')

In [26]:
df = pd.read_csv("../archivos/dt_enc_regis.csv", index_col = 0)
df1 = pd.read_csv("../archivos/dt_enc_casual.csv", index_col = 0)
df2 = pd.read_csv("../archivos/rf_enc_regis.csv", index_col = 0)
df3 = pd.read_csv("../archivos/rf_enc_casual.csv", index_col = 0)

In [27]:
resultados = pd.concat([df, df1, df2, df3], axis = 0)

In [28]:
resultados

,MAE,MSE,RMSE,R2,set,modelo
0,738.606226,883882.894708,940.150464,0.590524,test,Decision tree enc regis
1,634.326703,679143.733667,824.101774,0.727589,train,Decision tree enc regis
0,312.488861,215853.872501,464.600767,0.550701,test,Decision tree enc casual
1,305.857237,199621.160053,446.789839,0.573191,train,Decision tree enc casual
0,715.732121,698933.612176,836.022495,0.676205,test,Random Forest enc regis
1,694.639601,767116.344740,875.851782,0.692302,train,Random Forest enc regis
0,325.452573,201598.412748,448.997119,0.580374,test,Random Forest enc Casual
1,322.960756,197942.659479,444.907473,0.576780,train,Random Forest enc Casual
